Inception模块就是 四个卷积并行的层: 1. 一个1x1的卷积， 一个小的感受野进行卷积提取特征。 2. 一个1x1的卷积加上一个3x3的卷积， 1x1卷积降低输入的特征通道，减少参数数量， 3x3的卷积做一个较大感受野的卷积。 3. 一个1x1的卷积，加一个5x5的卷积。 4. 一个3x3的最大池化加上1x1的卷积，最大池化改变输入的特征排列， 1x1的卷积进行特征提取

In [1]:
import tensorflow as tf
batch_size = 64
import cifar10_input

In [2]:
images_trains, labels_trains = cifar10_input.distorted_inputs(data_dir="../cifar_data/cifar-10-batches-bin/",
                                                              batch_size=batch_size)

images_test, labels_test = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

In [3]:
import tensorflow.contrib.slim as slim

In [4]:
def inception(x,d0_1, d1_1, d1_3, d2_1, d2_5, d3_1, scope='inception', reuse=None):
    with tf.variable_scope(scope,reuse):
        #把slim.conv2d slim.max_pool2d 的默认参数放到slim的参数域中
        with slim.arg_scope([slim.conv2d, slim.max_pool2d], stride=1, padding='SAME'):
            with tf.variable_scope('branch0'):
                branch_0 = slim.conv2d(x, d0_1, [1,1], scope='conv_1x1')
            with tf.variable_scope('branch1'):
                branch_1 = slim.conv2d(x, d1_1, [1,1], scope='conv_1x1')
                branch_1 = slim.conv2d(branch_1, d1_3, [3,3], scope='conv_3x3')
            with tf.variable_scope('branch2'):
                branch_2 = slim.conv2d(x, d2_1, [1,1], scope='conv_1x1')
                branch_2 = slim.conv2d(branch_2, d2_5, [5,5], scope='conv_5x5')
            with tf.variable_scope('branch3'):
                branch_3 = slim.max_pool2d(x,[3,3], scope='max_pool')
                branch_3 = slim.conv2d(branch_3, d3_1, [1,1], scope='conv_1x1')
            
            net = tf.concat([branch_0, branch_1,branch_2,branch_3], axis=-1)
            
            return net

In [5]:
def googlenet(inputs, num_classes, reuse=None, is_training=None, verbose=False):
    with tf.variable_scope('googlenet',reuse=reuse):
        with slim.arg_scope([slim.batch_norm], is_training= is_training):
            with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], padding='SAME', stride=1):
                net = inputs
                with tf.variable_scope('block1'):
                    net = slim.conv2d(net, 64, [5,5], stride=2, scope='conv_5x5')
                    
                    if verbose:
                        print('block1 output : {}'.format(net.shape))
                with tf.variable_scope('block2'):
                    net = slim.conv2d(net, 64, [1,1], scope='conv_1x1')
                    net = slim.conv2d(net, 192, [3,3], scope='conv_3x3')
                    net = slim.max_pool2d(net, [3,3], stride=2, scope='max_pool')
                    
                    if verbose:
                        print('block2 output: {}'.format(net.shape))
                with tf.variable_scope('block3'):
                    net = inception(net, 64, 96, 128, 16, 32,32, scope='inception_1')
                    net = inception(net, 128, 128, 192,32, 94, 64, scope='inception_2')
                    net = slim.max_pool2d(net, [3,3], stride=2, scope='max_pool')
                
                    if verbose:
                        print('block3 output: {}'.format(net.shape))
                with tf.variable_scope('block4'):
                    net = inception(net, 192, 96, 208, 16, 48, 64, scope = 'inception_1')
                    net = inception(net, 160, 112, 224, 24, 64, 64, scope= 'inception_2')
                    net = inception(net, 128, 128, 256, 24, 64, 64, scope= 'inception_3')
                    net = inception(net, 112, 144, 288, 24, 64, 64, scope= 'inception_4')
                    net = inception(net, 256, 160, 320, 32, 128, 128, scope= 'inception_5')
                    net = slim.max_pool2d(net, [3,3], stride=2, scope='max_pool')
                    
                    if verbose:
                        print('block4 output: {}'.format(net.shape))
                with tf.variable_scope('block5'):
                    net = inception(net, 256, 160, 320, 32, 128, 128, scope='inception_1')
                    net = inception(net, 384, 182, 384, 48, 128, 128, scope='inception_2')
                    net = slim.avg_pool2d(net, [2,2], stride=2, scope='avg_pool')
                    if verbose:
                        print('block5 output: {}'.format(net.shape))
                
                with tf.variable_scope('classification'):
                    net = slim.flatten(net)
                    net = slim.fully_connected(net ,num_classes, activation_fn=None, 
                                               normalizer_fn=None, scope='light')
                    if verbose:
                        print('classification output: {}'.format(net.shape))
                return net

In [6]:
with slim.arg_scope([slim.conv2d], activation_fn = tf.nn.relu, normalizer_fn = slim.batch_norm) as sc:
    conv_scope = sc

is_training = tf.placeholder(tf.bool, name='is_training')

image_holder = tf.placeholder(shape=[batch_size,24,24,3], dtype=tf.float32)
label_holder = tf.placeholder(shape=[batch_size], dtype=tf.int32)

with slim.arg_scope(conv_scope):
    train_out = googlenet(image_holder,10, is_training=is_training, verbose=True)

train_out.shape

with tf.variable_scope('loss'):
    train_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=train_out,
                                                                       labels=label_holder, scope='train'))

with tf.variable_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1,output_type=tf.int32), 
                                                    label_holder), tf.float32))

lr = 1e-3

opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners()

for i in range(1000):
    image_bath, label_batch = sess.run([images_trains, labels_trains])
    _, loss1, acc1 = sess.run([train_op,train_loss,train_acc], feed_dict={image_holder: image_bath, label_holder: label_batch, is_training:True})
    if i % 10 ==0:
        print("Step%d  train loss %.6f train acc: %.6f" % (i+1, loss1, acc1))